In [ ]:
!pip install --upgrade git+https://github.com/EmGarr/kerod.git

# Setup training

In [ ]:
batch_size_per_gpu = 2
num_gpus = 8
batch_size = batch_size_per_gpu * num_gpus

# Download and prepare COCO/2017

Download and preprocess COCO/2017 to the following format (required by od networks):

```python
dataset = {
        'images' : A tensor of float32 and shape [1, height, widht, 3],
        'images_info': A tensor of float32 and shape [1, 2] ,
        'bbox': A tensor of float32 and shape [1, num_boxes, 4],
        'labels': A tensor of int32 and shape [1, num_boxes],
        'num_boxes': A tensor of int32 and shape [1, 1],
        'weights': A tensor of float32 and shape [1, num_boxes]
    }
```

If you need to download the dataset in a specific directory you can use the argument `data_dir` of `tfds.load`.

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from kerod.core.standard_fields import BoxField, DatasetField
from kerod.dataset.preprocessing import preprocess_coco_example
import functools


padded_shape = ({
  DatasetField.IMAGES: [None, None, 3],
  DatasetField.IMAGES_INFO: [2]
},
{
  BoxField.BOXES: [None, 4],
  BoxField.LABELS: [None],
  BoxField.NUM_BOXES: [1],
  BoxField.WEIGHTS: [None]
})

ds_train, ds_info = tfds.load(name="coco/2017", split="train", shuffle_files=True, with_info=True)
ds_train = ds_train.map(functools.partial(preprocess_coco_example, bgr=True),
                        num_parallel_calls=tf.data.experimental.AUTOTUNE)
# Filter example with no boxes after preprocessing
ds_train =  ds_train.filter(lambda x, y: tf.shape(y[BoxField.BOXES])[0] > 1)
ds_train =  ds_train.padded_batch(batch_size, padded_shape)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

ds_val = tfds.load(name="coco/2017", split="validation", shuffle_files=False)
ds_val = ds_val.map(functools.partial(preprocess_coco_example, horizontal_flip=False, bgr=True),
                      num_parallel_calls=tf.data.experimental.AUTOTUNE)
# Filter example with no boxes after preprocessing
ds_val =  ds_val.filter(lambda x, y: tf.shape(y[BoxField.BOXES])[0] > 1)
ds_val =  ds_val.padded_batch(batch_size, padded_shape)
ds_val = ds_val.prefetch(tf.data.experimental.AUTOTUNE)


# Load and train the network


In [ ]:
from kerod.core.standard_fields import BoxField
from kerod.core.learning_rate_schedule import LearningRateScheduler
from kerod.model import factory

from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint

# Number of classes of COCO
classes = ds_info.features['objects']['label'].names
num_classes = len(classes)

mirrored_strategy = tf.distribute.MirroredStrategy()
with mirrored_strategy.scope(): 
    model = factory.build_model(num_classes)
    base_lr = 0.02
    optimizer = tf.keras.optimizers.SGD(learning_rate=base_lr)
    model.compile(optimizer=optimizer, loss=None)
  
callbacks = [
    LearningRateScheduler(base_lr, num_gpus, epochs=[8, 10]),
    TensorBoard('log_tensorboard'),
    ModelCheckpoint('checkpoints/')
]

history = model.fit(ds_train, validation_data=ds_val, epochs=12, callbacks=callbacks)

In [ ]:
model.save_weights('awesome_model.h5')

In [ ]:
# Export a saved model for serving purposes
model.export_for_serving('serving')

## Coco evaluation


In [ ]:
model = factory.build_model(num_classes)
model.load_weights('awesome_model.h5')

In [ ]:
import tensorflow_datasets as tfds

ds_val, ds_info = tfds.load(name="coco/2017", split="validation", data_dir='dataset_tensorflow/', shuffle_files=False, with_info=True)
# category_ids basicaly map the index 0 the id
# e.g: 0 -> 1, 2 -> 3, 79 -> 90
category_ids = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 27, 28, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 67, 70, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 84, 85, 86, 87, 88, 89, 90]

### Super dirty but the evaluation works

#### Perform the analysis

In [ ]:
import json
import numpy as np
import tensorflow as tf

from kerod.core.standard_fields import DatasetField, BoxField             
from kerod.core.box_ops import convert_to_center_coordinates              
from kerod.dataset.preprocessing import resize_to_min_dim                 
                                                                       
results = []                                                           
                                                                       
for i, example in enumerate(ds_val): 
    print(i)
    # preprocess image 
    image = example['image'][:, :, ::-1]
    image = resize_to_min_dim(image, 800.0, 1333.0)         
    image_information = tf.cast(tf.shape(image)[:2], dtype=tf.float32) 
    inputs = {
        'images': tf.expand_dims(image, axis=0),
        'images_information':tf.expand_dims(image_information, axis=0)
    }
                                                                 
    # predict                                                          
    nmsed_boxes, nmsed_scores, nmsed_labels, valid_detections = model.predict_on_batch(inputs)
                                                                       
    # Post processing and append to coco results                       
    bbox = nmsed_boxes[0] * tf.tile(tf.expand_dims(tf.cast(example['image'].shape[:2], tf.float32)                    
        , axis=0), [1, 2])                   
    scores = nmsed_scores[0]                                           
    labels = nmsed_labels[0]                                           
    for i in range(valid_detections[0]):
        # Convert from [y_min, x_min, y_max, x_max] to coco format [x_min, y_min, w, h]
        sbox = bbox[i].numpy()
        sbox = [sbox[1], sbox[0], sbox[3] - sbox[1], sbox[2] - sbox[0]]
        res = {                                                        
                'image_id': int(example['image/id']),                       
                'category_id': category_ids[int(labels[i])],           
                'bbox': [round(float(c), 4) for c in sbox],
                'score': round(float(scores[i]), 4),                     
            }                                                          
        results.append(res)                                            
                                                   
                                                                       
with open('coco_results.json', 'w') as f:                              
    json.dump(results, f)

#### Install the coco library to compute the performances

In [ ]:
!wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip
!unzip annotations_trainval2017.zip
!pip install 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

#### Compute the performances

In [ ]:
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

with open('coco_results.json', 'r') as f:                              
    results = json.load(f)
coco = COCO('./annotations/instances_val2017.json')
ret = {}

cocoDt = coco.loadRes(results)
cocoEval = COCOeval(coco, cocoDt, 'bbox')
cocoEval.evaluate()
cocoEval.accumulate()
cocoEval.summarize()